In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
url_list = []

for i in list(range(1, 87)):
    url = "https://www.dermstore.com/skin-care.list?pageNumber=" + str(i)
    url_list.append(url)    

In [3]:
results_list = []
for url in url_list:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("li", class_ = "productListProducts_product")
    results_list.append(results)

In [4]:
results_string = str(results_list)
results_string = results_string.split("productBlock_productName")

In [13]:
product_list = []
for ele in results_string:
    ele = ele.split("data")
    name = list(set([string for string in ele if "-product-name" in string]))
    brand = list(set([string for string in ele if "-product-brand" in string]))
    price = list(set([string for string in ele if "-product-price" and "$" in string]))
    url = list(set([string for string in ele if "href" and ".html" in string]))
    dict_i = {"product_name": name, "brand": brand, "price": price, "url": url}
    product_list.append(dict_i)

In [14]:
products_df = pd.DataFrame(product_list)
products_df = products_df[products_df["product_name"].map(lambda d: len(d)) > 0].reset_index()
products_df = products_df.applymap(lambda x: x[0] if isinstance(x, list) else x)

products_df["product_name"] = products_df["product_name"].str.replace("-product-name=", "")
products_df["product_name"] = products_df["product_name"].str.replace('"', "")

products_df["brand"] = products_df["brand"].str.replace("-product-brand=", "")
products_df["brand"] = products_df["brand"].str.replace('"', "")

products_df["price"] = products_df["price"].str.replace("-product-price=", "")
products_df["price"] = products_df["price"].str.replace('"', "")

products_df["url"] = products_df["url"].apply(lambda x: x.split("href")[-1])
products_df["url"] = products_df["url"].str.replace("=", "")
products_df["url"] = products_df["url"].str.replace('"', "")
products_df["url"] = products_df["url"].str.replace(">\n<div classproductBlock_title>\n<h3 class", "")
products_df["url"] = products_df["url"].str.replace(">\n<div classproductBlock_imageContainer ", "")
products_df["url"] = "https://www.dermstore.com" + products_df["url"]

products_df = products_df.drop(columns = "index")

In [15]:
products_df

,product_name,brand,price,url
0,ELEMIS Pro-Collagen Cleansing Balm (100 g.),Elemis,$68.00,https://www.dermstore.com/elemis-pro-collagen-...
1,VIRTUE Healing Oil (1.7 fl. oz.),VIRTUE,$44.00,https://www.dermstore.com/virtue-healing-oil-5...
2,VIRTUE Flourish Nightly Intensive Hair Growth ...,VIRTUE,$134.00,https://www.dermstore.com/virtue-flourish-nigh...
3,Augustinus Bader The Rich Cream 50ml,Augustinus Bader,$290.00,https://www.dermstore.com/augustinus-bader-the...
4,SkinCeuticals C E Ferulic (1 fl. oz.),SkinCeuticals,$182.00,https://www.dermstore.com/skinceuticals-c-e-fe...
...,...,...,...,...
4330,LightStim LightStim for Acne (1 kit),LightStim,$169.00,https://www.dermstore.com/lightstim-for-acne-l...
4331,LightStim LightStim PhotoSerum (1 fl. oz.),LightStim,$80.00,https://www.dermstore.com/lightstim-photoserum...
4332,FOREO BEAR mini 1 piece,FOREO,$219.00,https://www.dermstore.com/foreo-bear-mini-faci...
4333,Murad Hydrate Trial Kit 2.83 fl. oz. - $58 Value,Murad,$36.00,https://www.dermstore.com/murad-hydrate-trial-...


In [16]:
products_df.to_csv(r"data/dermstore_basic.csv")

In [81]:
products_df = pd.read_csv("data/dermstore_basic.csv")

In [82]:
products_df = products_df.drop(index = [3057, 3087, 3153, 3176])

In [83]:
url_list2 = products_df["url"].to_list()

In [113]:
results_list = []
for url in url_list2:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("div", class_ = "athenaProductPage_breakpoint-lg_productDescription")
    results_list.append(results)
    
results_list = [str(ele) for ele in results_list]

In [ ]:
ingredients_list = []

for element in results_list:
    ele = element.replace('"', "").split("<div class=athenaProductPageSynopsisContent><p>")
    ele = ele[-1]
    ingredients = ele
    dict_i = {"ingredients": ingredients}
    ingredients_list.append(dict_i)

ing_df = pd.DataFrame(ingredients_list)
ing_df["url"] = url_list2

In [114]:
ing_df

,ingredients,url
0,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ca...",https://www.dermstore.com/elemis-pro-collagen-...
1,<strong>Active:</strong> Alpha Keratin 60ku®.<...,https://www.dermstore.com/virtue-healing-oil-5...
2,Please see individual products for ingredients...,https://www.dermstore.com/virtue-flourish-nigh...
3,"Aqua (Water), Coco-Caprylate/Caprate, Helianth...",https://www.dermstore.com/augustinus-bader-the...
4,"Aqua/Water/Eau, Ethoxydiglycol, Ascorbic Acid,...",https://www.dermstore.com/skinceuticals-c-e-fe...
...,...,...
4326,"36-LED Light Device, 9-Volt Power Cord, Goggle...",https://www.dermstore.com/lightstim-for-acne-l...
4327,"Water, Cyclopentasiloxane, Cyclohexasiloxane, ...",https://www.dermstore.com/lightstim-photoserum...
4328,"Body-Safe Silicone, ABS, PC, Chrome-Plated Zin...",https://www.dermstore.com/foreo-bear-mini-faci...
4329,Please see individual products for ingredients...,https://www.dermstore.com/murad-hydrate-trial-...


In [127]:
products_df1 = products_df.copy()

In [129]:
products_df1.merge(ing_df)

,Unnamed: 0,product_name,brand,price,url,ingredients
0,0,ELEMIS Pro-Collagen Cleansing Balm (100 g.),Elemis,$68.00,https://www.dermstore.com/elemis-pro-collagen-...,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ca..."
1,0,ELEMIS Pro-Collagen Cleansing Balm (100 g.),Elemis,$68.00,https://www.dermstore.com/elemis-pro-collagen-...,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ca..."
2,0,ELEMIS Pro-Collagen Cleansing Balm (100 g.),Elemis,$68.00,https://www.dermstore.com/elemis-pro-collagen-...,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ca..."
3,0,ELEMIS Pro-Collagen Cleansing Balm (100 g.),Elemis,$68.00,https://www.dermstore.com/elemis-pro-collagen-...,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ca..."
4,0,ELEMIS Pro-Collagen Cleansing Balm (100 g.),Elemis,$68.00,https://www.dermstore.com/elemis-pro-collagen-...,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ca..."
...,...,...,...,...,...,...
27610,4330,LightStim LightStim for Acne (1 kit),LightStim,$169.00,https://www.dermstore.com/lightstim-for-acne-l...,"36-LED Light Device, 9-Volt Power Cord, Goggle..."
27611,4331,LightStim LightStim PhotoSerum (1 fl. oz.),LightStim,$80.00,https://www.dermstore.com/lightstim-photoserum...,"Water, Cyclopentasiloxane, Cyclohexasiloxane, ..."
27612,4332,FOREO BEAR mini 1 piece,FOREO,$219.00,https://www.dermstore.com/foreo-bear-mini-faci...,"Body-Safe Silicone, ABS, PC, Chrome-Plated Zin..."
27613,4333,Murad Hydrate Trial Kit 2.83 fl. oz. - $58 Value,Murad,$36.00,https://www.dermstore.com/murad-hydrate-trial-...,Please see individual products for ingredients...


In [130]:
ing_df

,ingredients,url
0,"Prunus Amygdalus Dulcis (Sweet Almond) Oil, Ca...",https://www.dermstore.com/elemis-pro-collagen-...
1,<strong>Active:</strong> Alpha Keratin 60ku®.<...,https://www.dermstore.com/virtue-healing-oil-5...
2,Please see individual products for ingredients...,https://www.dermstore.com/virtue-flourish-nigh...
3,"Aqua (Water), Coco-Caprylate/Caprate, Helianth...",https://www.dermstore.com/augustinus-bader-the...
4,"Aqua/Water/Eau, Ethoxydiglycol, Ascorbic Acid,...",https://www.dermstore.com/skinceuticals-c-e-fe...
...,...,...
4326,"36-LED Light Device, 9-Volt Power Cord, Goggle...",https://www.dermstore.com/lightstim-for-acne-l...
4327,"Water, Cyclopentasiloxane, Cyclohexasiloxane, ...",https://www.dermstore.com/lightstim-photoserum...
4328,"Body-Safe Silicone, ABS, PC, Chrome-Plated Zin...",https://www.dermstore.com/foreo-bear-mini-faci...
4329,Please see individual products for ingredients...,https://www.dermstore.com/murad-hydrate-trial-...


In [134]:
products_df = products_df.drop(columns = "Unnamed: 0")

In [144]:
merged_df = pd.merge(products_df, ing_df, on='url')

In [145]:
merged_df = merged_df.drop_duplicates(subset = "product_name")

In [148]:
merged_df.to_csv(r"data/ingredients_skincare_dermstore.csv")

In [169]:
# read in data and clean ingredients
df = pd.read_csv("data/ingredients_skincare_dermstore.csv").drop(columns = "Unnamed: 0")
for index, row in df.iterrows():
    row["ingredients"] = row["ingredients"].split("</p>")
    row["ingredients"] = row["ingredients"][:-1]

# create active and inactive ingredient rows
df["active_ingredients"] = ""
df["inactive_ingredients"] = ""    
for index, row in df.iterrows():
    if len(row["ingredients"]) > 1: 
        row["active_ingredients"] = row["ingredients"][0]
        row["inactive_ingredients"] = row["ingredients"][1]
        row["active_ingredients"] = row["active_ingredients"].replace("<strong>Active:</strong> ", "")
        row["active_ingredients"] = row["active_ingredients"][:-1]
        row["inactive_ingredients"] = row["inactive_ingredients"].replace("<p><strong>Inactive:</strong> ", "")
        row["inactive_ingredients"] = row["inactive_ingredients"].replace("<p><strong>Inactive: </strong>", "")
        row["ingredients"] = row["active_ingredients"] + ", " + row["inactive_ingredients"]
    else:
        row["ingredients"] = str(row["ingredients"])[2:-2]

In [ ]:
# remove rows with incorrect ingredient data from dataframe
df = df.loc[df["ingredients"].str.contains("100%") == False]
df = df.loc[df["ingredients"].str.contains("Please") == False]
df = df.loc[df["ingredients"].str.contains("individual") == False]
df = df.loc[df["ingredients"].str.contains("Device") == False]
df = df.loc[df["ingredients"].str.contains("LED") == False]
df = df.loc[df["ingredients"].str.contains("Glass") == False]
df = df.loc[df["ingredients"].str.contains("N/A") == False]
df = df.loc[df["ingredients"].str.contains("Fibers") == False]
df = df.loc[df["ingredients"].str.contains("Disepense") == False]
df = df.loc[df["ingredients"].str.contains(",")]
df = df.loc[df["ingredients"].str.contains("<xml>") == False]
df = df.loc[df["ingredients"].str.contains("<div class=athenaProductPage_breakpoint") == False]

no_water_list = []

for index, row in df.iterrows():
    if "Water" not in row["ingredients"]:
        if "water" not in row["ingredients"]:
            if "Acid" not in row["ingredients"]:
                if "Oil" not in row["ingredients"]:
                    if "oil" not in row["ingredients"]:
                        if "AQUA" not in row["ingredients"]:
                            if "Aqua" not in row["ingredients"]:
                                if "gum" not in row["ingredients"]:
                                    if "ingredient" not in row["ingredients"]:
                                        if "Ingredient" not in row["ingredients"]:
                                            if "Extract" not in row["ingredients"]:
                                                no_water_list.append(index)
no_water = df[["ingredients"]]
no_water = no_water.loc[no_water_list]
no_water = no_water.reset_index()
bad_indexes = no_water["index"].to_list()
df = df.loc[~df.index.isin(bad_indexes)]

In [277]:
df.to_csv(r"data/ingredients_skincare_dermstore.csv")